# Amazon SageMaker Autopilot Candidate Definition Notebook

This notebook was automatically generated by the AutoML job **xgb-c1-2020-04-11-22-09-21-981**.
This notebook allows you to customize the candidate definitions and execute the SageMaker Autopilot workflow.

The dataset has **21** columns and the column named **y** is used as
the target column. This is being treated as a **BinaryClassification** problem. The dataset also has **2** classes.
This notebook will build a **[BinaryClassification](https://en.wikipedia.org/wiki/Binary_classification)** model that
**maximizes** the "**ACCURACY**" quality metric of the trained models.
The "**ACCURACY**" metric provides the percentage of times the model predicted the correct class.

As part of the AutoML job, the input dataset has been randomly split into two pieces, one for **training** and one for
**validation**. This notebook helps you inspect and modify the data transformation approaches proposed by Amazon SageMaker Autopilot. You can interactively
train the data transformation models and use them to transform the data. Finally, you can execute a multiple algorithm hyperparameter optimization (multi-algo HPO)
job that helps you find the best model for your dataset by jointly optimizing the data transformations and machine learning algorithms.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>
Look for sections like this for recommended settings that you can change.
</div>


---

## Contents

1. [Sagemaker Setup](#Sagemaker-Setup)
    1. [Downloading Generated Candidates](#Downloading-Generated-Modules)
    1. [SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration](#SageMaker-Autopilot-Job-and-Amazon-Simple-Storage-Service-(Amazon-S3)-Configuration)
1. [Candidate Pipelines](#Candidate-Pipelines)
    1. [Generated Candidates](#Generated-Candidates)
    1. [Selected Candidates](#Selected-Candidates)
1. [Executing the Candidate Pipelines](#Executing-the-Candidate-Pipelines)
    1. [Run Data Transformation Steps](#Run-Data-Transformation-Steps)
    1. [Multi Algorithm Hyperparameter Tuning](#Multi-Algorithm-Hyperparameter-Tuning)
1. [Model Selection and Deployment](#Model-Selection-and-Deployment)
    1. [Tuning Job Result Overview](#Tuning-Job-Result-Overview)
    1. [Model Deployment](#Model-Deployment)

---

## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=v1.43.4`
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **xgb-c1-2020-04-11-22-09-21-981-artifacts** folder.

In [2]:
!mkdir -p xgb-c1-2020-04-11-22-09-21-981-artifacts
!aws s3 sync s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/sagemaker-automl-candidates/pr-1-4b34747772154bc0b1d5924ce17aa391a9917a53f2084af6b9542512b9/generated_module xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module --only-show-errors
!aws s3 sync s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/sagemaker-automl-candidates/pr-1-4b34747772154bc0b1d5924ce17aa391a9917a53f2084af6b9542512b9/notebooks/sagemaker_automl xgb-c1-2020-04-11-22-09-21-981-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("xgb-c1-2020-04-11-22-09-21-981-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

In [3]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'xgb-c1-2020-04-11-22-09-21-981'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981',
    'data_transformer_image_repo_version': '0.1.0-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '0.90-1-cpu-py3', 'linear-learner': 'latest'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'xgb-c1-202-notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::262002448484:role/service-role/AmazonSageMaker-ExecutionRole-20190606T095855',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::262002448484:role/service-role/AmazonSageMaker-ExecutionRole-20190606T095855</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>xgb-c1-2020-04-11-22-09-21-981</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>xgb-c1-202-notebook-run-11-22-41-31</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/xgb-c1-202-notebook-run-11-22-41-31</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/xgb-c1-202-notebook-run-11-22-41-31/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/xgb-c1-202-notebook-run-11-22-41-31/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://sagemaker-us-west-1-262002448484/sagemaker/DEMO-automl-dm/output/xgb-c1-2020-04-11-22-09-21-981/xgb-c1-202-notebook-run-11-22-41-31/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

In [4]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **10** machine learning
pipeline(s) that use **2** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [5]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp1-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [6]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp1",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp2-linear-learner](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [7]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp3-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [8]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp3",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp4-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [9]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp4",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp5-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [10]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp5",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp6-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [11]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp6",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp7-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp7.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [12]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp7",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp8-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp8.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [13]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp8",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp9-xgboost](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp9.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [14]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp9",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [15]:
automl_interactive_runner.display_candidates()

Candidate Name,Algorithm,Feature Transformer
dpp0-xgboost,xgboost,dpp0.py
dpp1-xgboost,xgboost,dpp1.py
dpp2-linear-learner,linear-learner,dpp2.py
dpp3-xgboost,xgboost,dpp3.py
dpp4-xgboost,xgboost,dpp4.py
dpp5-xgboost,xgboost,dpp5.py
dpp6-xgboost,xgboost,dpp6.py
dpp7-xgboost,xgboost,dpp7.py
dpp8-xgboost,xgboost,dpp8.py
dpp9-xgboost,xgboost,dpp9.py


The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](xgb-c1-2020-04-11-22-09-21-981-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

In [16]:
automl_interactive_runner.fit_data_transformers(parallel_jobs=7)

2020-04-11 23:27:23,537 INFO root: [Worker_0:dpp0-xgboost]Executing step: train_data_transformer
2020-04-11 23:27:23,541 INFO root: [Worker_5:dpp5-xgboost]Executing step: train_data_transformer
2020-04-11 23:27:23,770 INFO sagemaker: Creating training-job with name: xgb-c1-202-notebook-run-11-22-41-31-dpp5-train-11-23-27-22
2020-04-11 23:27:23,783 INFO sagemaker: Creating training-job with name: xgb-c1-202-notebook-run-11-22-41-31-dpp0-train-11-23-27-22

2020-04-11 23:27:23 Starting - Starting the training job
2020-04-11 23:27:24 Starting - Starting the training job2020-04-11 23:27:24,542 INFO root: [Worker_3:dpp3-xgboost]Executing step: train_data_transformer
2020-04-11 23:27:24,543 INFO root: [Worker_6:dpp6-xgboost]Executing step: train_data_transformer
2020-04-11 23:27:24,618 INFO sagemaker: Creating training-job with name: xgb-c1-202-notebook-run-11-22-41-31-dpp6-train-11-23-27-22
2020-04-11 23:27:24,644 INFO sagemaker: Creating training-job with name: xgb-c1-202-notebook-run-11-22

2020-04-11 23:30:33 Completed - Training job completed
...2020-04-11 23:30:39,085 INFO root: [Worker_3:dpp3-xgboost]Executing step: create_transformer_model
2020-04-11 23:30:39,110 INFO sagemaker: Creating model with name: xgb-c1-202-notebook-run-11-22-41-31-dpp3-train-11-23-27-22
...2020-04-11 23:30:41,433 INFO root: [Worker_3:dpp3-xgboost]Executing step: perform_data_transform
2020-04-11 23:30:41,434 INFO sagemaker: Creating transform job with name: xgb-c1-202-notebook-run-11-22-41-31-dpp3-transform-11-23-27-22
.....................................................................................................................................................................................................................!
2020-04-11 23:33:14,177 INFO root: Successfully fit data transformer for dpp6-xgboost
...2020-04-11 23:33:17,180 INFO root: [Worker_6:dpp7-xgboost]Executing step: train_data_transformer
2020-04-11 23:33:17,251 INFO sagemaker: Creating training-job with name: xgb-c1-

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible trasformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [17]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:accuracy',
    'linear-learner': 'validation:binary_classification_accuracy',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'binary:hinge',
    },
    'linear-learner': {
        'predictor_type': 'binary_classifier',
        'loss': 'logistic',
        'mini_batch_size': 800,
        'binary_classifier_model_selection_criteria': 'loss_function',
        'num_models': 1,
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [18]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(2, 512, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 32, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'wd': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'l1': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-5, 1.0, scaling_type='Logarithmic'),
        'positive_example_weight_mult': CategoricalParameter(['balanced', '0.01', '1', '100']),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [19]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

Below you prepare the inputs data to the multi-algo tuner:

In [20]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

In [21]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Maximize',
    max_parallel_jobs=7,
    max_jobs=250,
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [22]:
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2020-04-11 23:41:56,740 INFO root: _TuningJob.start_new!!!
2020-04-11 23:41:56,745 INFO sagemaker: Creating hyperparameter tuning job with name: xgb-c1-202-notebook--200411-2341


Tuning Job xgb-c1-202-notebook--200411-2341 started, please track the progress from [here](https://us-west-1.console.aws.amazon.com/sagemaker/home?region=us-west-1#/hyper-tuning-jobs/xgb-c1-202-notebook--200411-2341)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [23]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobDefinitionName,TrainingJobName,TrainingJobStatus,TrainingStartTime,alpha,colsample_bytree,eta,gamma,l1,lambda,learning_rate,max_depth,min_child_weight,num_round,positive_example_weight_mult,subsample,wd
123,0.918978,65.0,2020-04-12 00:42:46+00:00,dpp6-xgboost,xgb-c1-202-notebook--200411-2341-127-733d3050,Completed,2020-04-12 00:41:41+00:00,0.009812,0.873918,0.028251,6.118777,NaN,0.024372,NaN,28.0,0.849376,147.0,NaN,0.530783,NaN
155,0.918722,70.0,2020-04-12 00:27:13+00:00,dpp8-xgboost,xgb-c1-202-notebook--200411-2341-095-57108b48,Completed,2020-04-12 00:26:03+00:00,0.000005,0.884270,0.216711,21.586894,NaN,1.577044,NaN,21.0,0.847793,204.0,NaN,0.693158,NaN
32,0.917827,89.0,2020-04-12 01:28:06+00:00,dpp6-xgboost,xgb-c1-202-notebook--200411-2341-218-7b5ee65d,Completed,2020-04-12 01:26:37+00:00,0.259029,0.979356,0.108421,15.715688,NaN,0.000039,NaN,29.0,0.015192,378.0,NaN,0.553868,NaN
173,0.917827,99.0,2020-04-12 00:18:15+00:00,dpp8-xgboost,xgb-c1-202-notebook--200411-2341-077-afcfcd58,Completed,2020-04-12 00:16:36+00:00,0.017566,0.628456,0.008520,23.284713,NaN,0.000011,NaN,28.0,0.000002,441.0,NaN,0.799170,NaN
120,0.917572,79.0,2020-04-12 00:44:17+00:00,dpp8-xgboost,xgb-c1-202-notebook--200411-2341-130-00168bce,Completed,2020-04-12 00:42:58+00:00,0.000003,0.898442,0.080735,15.217189,NaN,0.240728,NaN,32.0,0.000068,262.0,NaN,0.668071,NaN
13,0.917316,60.0,2020-04-12 01:37:54+00:00,dpp6-xgboost,xgb-c1-202-notebook--200411-2341-237-91f92591,Completed,2020-04-12 01:36:54+00:00,0.000011,0.948516,0.568853,24.269321,NaN,0.001695,NaN,3.0,0.000002,69.0,NaN,0.755431,NaN
9,0.917316,79.0,2020-04-12 01:39:28+00:00,dpp6-xgboost,xgb-c1-202-notebook--200411-2341-241-6925ce3a,Completed,2020-04-12 01:38:09+00:00,0.113147,0.703614,0.021005,0.000102,NaN,0.008775,NaN,9.0,0.063952,481.0,NaN,0.899242,NaN
204,0.917188,51.0,2020-04-12 00:02:38+00:00,dpp8-xgboost,xgb-c1-202-notebook--200411-2341-046-2dd0cff3,Completed,2020-04-12 00:01:47+00:00,0.000217,0.835274,0.315955,31.027451,NaN,0.000002,NaN,7.0,0.000001,17.0,NaN,0.840808,NaN
55,0.917188,74.0,2020-04-12 01:16:04+00:00,dpp6-xgboost,xgb-c1-202-notebook--200411-2341-195-26f03e7b,Completed,2020-04-12 01:14:50+00:00,0.000002,0.767375,0.194379,38.928994,NaN,0.003743,NaN,31.0,0.000001,253.0,NaN,0.772319,NaN
202,0.917061,49.0,2020-04-12 00:02:56+00:00,dpp8-xgboost,xgb-c1-202-notebook--200411-2341-048-28dbcf51,Completed,2020-04-12 00:02:07+00:00,0.016889,0.987124,0.348582,0.000011,NaN,0.664581,NaN,3.0,0.001480,104.0,NaN,0.769192,NaN


The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [24]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

Best Multi Algorithm HPO training job name is xgb-c1-202-notebook--200411-2341-127-733d3050


### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.
3. Inverse Label Transformer Container: a container that converts numerical intermediate prediction value back to non-numerical label value.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

In [25]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(env={'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT':"text/csv"})

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))

pipeline_model = PipelineModel(
    name="AutoML-{}".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name),
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

2020-04-12 03:43:31,467 INFO root: Chosen Data Processing pipeline candidate name is dpp6-xgboost
2020-04-11 23:29:46 Starting - Preparing the instances for training
2020-04-11 23:29:46 Downloading - Downloading input data
2020-04-11 23:29:46 Training - Training image download completed. Training in progress.
2020-04-11 23:29:46 Uploading - Uploading generated training model
2020-04-11 23:29:46 Completed - Training job completed2020-04-11 23:29:29,161 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-04-11 23:29:29,163 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-11 23:29:29,173 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-04-11 23:29:29,434 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-04-11 23:29:29,434 sagemaker-containers INFO     Generating setup.cfg
2020-04-11 23:29:29,434 sagemaker-containers 

[141]#011train-error:0.050091#011validation-error:0.081406#011train-accuracy:0.949909#011validation-accuracy:0.918594
[142]#011train-error:0.050059#011validation-error:0.080767#011train-accuracy:0.949941#011validation-accuracy:0.919233
[143]#011train-error:0.049804#011validation-error:0.080639#011train-accuracy:0.950196#011validation-accuracy:0.919361
[144]#011train-error:0.049548#011validation-error:0.080639#011train-accuracy:0.950452#011validation-accuracy:0.919361
[145]#011train-error:0.04926#011validation-error:0.081022#011train-accuracy:0.95074#011validation-accuracy:0.918978
[146]#011train-error:0.049133#011validation-error:0.081022#011train-accuracy:0.950867#011validation-accuracy:0.918978
Training seconds: 65
Billable seconds: 65
2020-04-11 23:29:46 Starting - Preparing the instances for training
2020-04-11 23:29:46 Downloading - Downloading input data
2020-04-11 23:29:46 Training - Training image download completed. Training in progress.
2020-04-11 23:29:46 Uploading - Uploadi

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [ ]:
pipeline_model.deploy(initial_instance_count=1,
                      instance_type='ml.m5.2xlarge',
                      endpoint_name=pipeline_model.name,
                      wait=True)

2020-04-12 03:43:38,434 INFO sagemaker: Creating model with name: AutoML-xgb-c1-202-notebook-run-11-22-41-31
2020-04-12 03:43:39,387 INFO sagemaker: Creating endpoint with name AutoML-xgb-c1-202-notebook-run-11-22-41-31
-

Congratulations! Now you could visit the sagemaker
[endpoint console page](https://us-west-1.console.aws.amazon.com/sagemaker/home?region=us-west-1#/endpoints) to find the deployed endpoint (it'll take a few minutes to be in service).

<div class="alert alert-warning">
    <strong>To rerun this notebook, delete or change the name of your endpoint!</strong> <br>
If you rerun this notebook, you'll run into an error on the last step because the endpoint already exists. You can either delete the endpoint from the endpoint console page or you can change the <code>endpoint_name</code> in the previous code block.
</div>